# Proposito
En este notebook estan los pasos a tomar para la etapa de **estructuracion** del proceso de **Data Wrangling**. Este proceso recibe como entrada los datos obtenidos en la etapa anterior: **descubrimiento**.

Para este ejemplo, recibiremos como insumo un archivo CSV llamado `raw_data.csv`.

Finalmente, no olvidemos la necesidad del negocio. Necesitamos dar respuesta a la pregunta ¿Que grupo etario trae mas mal riesgo al banco?

# Objetivos
Al finalizar este notebook deberemos de haber:

✅ Verificar que los datos sigan una estructura definida.

✅ Verificar la presencia o ausencia de valores faltantes.

✅ Verificar la presencia o ausencia de valores extraños.

In [1]:
import pandas as pd
from pandas import DataFrame

In [2]:
df = pd.read_csv(r"../data/raw_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0.1       4846 non-null   int64  
 1   Duration           4699 non-null   float64
 2   Checking account   2732 non-null   object 
 3   Credit amount      4576 non-null   object 
 4   Purpose            4686 non-null   object 
 5   Job                4635 non-null   float64
 6   Job                4635 non-null   float64
 7   Sex                4617 non-null   object 
 8   Sex                4617 non-null   object 
 9   Checking account   2732 non-null   object 
 10  Purpose            4686 non-null   object 
 11  csv                0 non-null      float64
 12  data_source        4844 non-null   object 
 13  Saving accounts    3788 non-null   object 
 14  Housing            4622 non-null   object 
 15  Risk               4715 non-null   object 
 16  Age                4705 

# Los hallazgos
Podemos determinar a simple vista multiples cosas:
1. Los datos no siguen una estructura definida, esperabamos que multiples columnas fueran de un tipo de dato determinado como `float64`, `int64` o `string` y en lugar de ello estamos obteniendo `object`. Esto es un indicio de la presencia de valores extraños dentro de estas columnas.

2. Tenemos un problema, hay columnas duplicadas en nuestro set de datos. 

3. Una gran cantidad de columnas tienen valores nulos. 

Ataquemos uno por uno estos problemas, empezare por el ulitmo hasta llegar al primero. 

No existe una razon concreta para este decision, se podrian atacar estos problemas en cualquier orden deseado por el lector.

# Atacando el problema de los valores nulos
Hay multiples formas de abordar esto, una de ellas es eliminando las filas cuyos valores sean nulos, eliminando las columnas que tengan muchos valores nulos o simplemente dejandolos 🙃. 

Primero, determinemos si hay columnas con muchos valores nulos. Pero... antes de poderlo hacer, tenemos que determinar si nuestro set de datos tiene otras representaciones comunes para los valores nulos.

In [3]:
def find_weird_NAN_representations(df: DataFrame) -> None:
    """Esta funcion revisa si hay otras representaciones comunes para los valores nulos dentro del
    DataFrame.

    Args:
        df (DataFrame): DataFrame que quiero revisar para obtener representaciones
        comunes de NAN.
    """
    nan_representations: list[str] = ["NA", "N/A", "null", "NULL", "nan", "NaN", "", " ", "?"]
    columns: list[str] = list()
    for column in df.columns:
        if len(df[df[column].isin(nan_representations)]) > 0:
            columns.append(column)
    print(
        f"NAN representation found for columns {columns}"
        if len(columns) > 0
        else "No NAN representation found in columns"
    )

In [4]:
find_weird_NAN_representations(df)

No NAN representation found in columns


In [5]:
def find_NAN_no_NAN_match_column_length(df: DataFrame):
    """Una ultima inspeccion que podemos realizar es que la suma de mis valores no nulos y mis valores nulos
    sean la longitud que tiene la columna, de lo contrario podremos saber que hay alguna otra representacion
    que estamos pasando alto.

    Args:
        df (DataFrame): DataFrame que quiero revisar.
    """
    columns: list[str] = list()
    for column in df.columns:
        if len(df[pd.isna(df[column])]) + len(df[df[column].notna()]) != len(df[column]):
            columns.append(column)
    print(
        f"NAN + not NAN check invalid for columns: {columns}"
        if len(columns) > 0
        else "NAN + not NAN check valid for all columns"
    )

In [6]:
find_NAN_no_NAN_match_column_length(df)

NAN + not NAN check valid for all columns


Perfecto, ya que vimos que no hay mas representacion para los nulos revisemos el % de nulos por columna.

In [7]:
df.isnull().sum() / len(df) * 100

Unnamed: 0.1           0.000000
Duration               3.033430
Checking account      43.623607
Credit amount          5.571605
Purpose                3.301692
Job                    4.354106
Job                    4.354106
Sex                    4.725547
Sex                    4.725547
Checking account      43.623607
Purpose                3.301692
csv                  100.000000
data_source            0.041271
Saving accounts       21.832439
Housing                4.622369
Risk                   2.703260
Age                    2.909616
value                  0.041271
Unnamed: 0             3.322328
dtype: float64

La columna `csv` no contiene nada mas que valores nulos, podriamos eliminarla sin problemas.

# Atacando el problema de las columnas duplicadas
¿Realmente son columnas duplicadas o solo sus nombres son iguales? Algunas veces pensamos que son dos columnas duplicadas solo por que tienen el mismo nombre, sin embargo, podrian no tener los mismos datos. 

Validemos esto. 

In [8]:
df["Sex"].equals(df["Sex "])

True

In [9]:
df["Job"].equals(df["Job "])

True

In [10]:
df["Purpose"].equals(df["Purpose "])

True

In [11]:
df["Checking account"].equals(df["Checking account "])

True

Como podemos obervar si que son columnas identicas, con los mismos tipos de datos.

# Atacando el problema de la estructura de los datos
Cuando hallamos un dato tipo `object` en el DataFrame es por que hay algun valor extraño dentro de la columna que no permite a Pandas detectar correctamente el tipo de dato de la misma. 

Veamos como hallarlos.

In [12]:
def find_weird_values_for_columns(df: DataFrame, numeric_columns: list[str]) -> DataFrame:
    """Esta funcion halla valores no numericos dentro de las columnas numericas especificadas.

    Args:
        df (DataFrame): DataFrame que contiene las columnas numericas a buscar por valores extraños.
        numeric_columns (list[str]): Lista de columnas a buscar.

    Returns:
        DataFrame: Nombre de la columna, y lista de valores extraños hallados.
    """
    copy_df: DataFrame = df.copy()
    columns: list[str] = list()
    masks: list = list()
    for column in numeric_columns:
        copy_df[column] = copy_df[column].astype(
            str
        )  # convert to string temporarily for checking weird values
        mask = ~copy_df[column].str.match(r"^-?\d+\.?\d*$", na=True)
        if len(df[mask][column].unique()) > 0:
            columns.append(column)
            masks.append(df[mask][column].unique())

    weird_values: DataFrame = DataFrame({"Column": columns, "Weird_Values": masks})
    return weird_values

In [13]:
find_weird_values_for_columns(df, ["Duration", "Credit amount", "Age"])

,Column,Weird_Values
0,Duration,[nan]
1,Credit amount,"[nan, dfas, qwretryet, ttqweyuet]"
2,Age,"[nan, hgd]"


Ahora intentemos hallar valores no esperados para columnas no numericas

In [14]:
def find_unexpected_values(df: DataFrame, expected_values: dict[str, str]) -> DataFrame:
    """Esta funcion halla valores extraños dado un conjunto de valores esperados x columna.

    Args:
        df (DataFrame): DataFrame que contiene las columnas numericas a buscar por valores extraños.
        expected_values (dict[str, str]): Conjunto de valores esperados x columna.

    Returns:
        DataFrame: Nombre de la columna, y lista de valores extraños hallados.
    """
    categories_to_review: list[str] = list(
        set(df.columns).intersection(set(expected_values.keys()))
    )
    if not categories_to_review:
        return DataFrame(columns=["Column", "Unexpected_Values"])

    unexpected_values: dict = {
        col: df.loc[~df[col].isin(expected_values[col]), col].unique().tolist()
        for col in categories_to_review
        if (~df[col].isin(expected_values[col])).any()
    }

    return DataFrame(
        {"Column": unexpected_values.keys(), "Unexpected_Values": unexpected_values.values()}
    )

In [15]:
expected_values = {
    "Housing": ["own", "rent", "free"],
    "Sex": ["male", "female"],
    "Purpose": [
        "car",
        "radio/TV",
        "furniture/equipment",
        "business",
        "education",
        "repairs",
        "domestic appliances",
        "vacation/others",
    ],
}

find_unexpected_values(df, expected_values)

,Column,Unexpected_Values
0,Purpose,"[nan, 6, 3, 56, 356]"
1,Housing,"[nan, 563, 43, 356]"
2,Sex,"[nan, 353546]"


# Conclusiones
Pudimos alcanzar cada uno de los objetivos propuestos al inicio de este notebook.
## Verificar que los datos sigan una estructura definida y la presencia o ausencia de los mismos.
Pudimos observar que algunas columnas numericas y de texto tienen valores extraños o no esperados.
## Verificar la presencia o ausencia de valores faltantes.
Validamos el % de nulos x cada una de las columnas determinando que una de ellas puede ser borrada.
